In [ ]:
import json
import os
import subprocess
import time

In [1]:
materials = {
    'Material_1': [
        'Steel', 'Copper', 'Aluminum', 'Aluminum', 'Brass', 'Cadmium', 'Cadmium',
        'Cast Iron', 'Chromium', 'Copper', 'Copper', 'Copper', 'Glass', 'Glass',
        'Glass', 'Graphite', 'Graphite', 'Nickel', 'Nickel', 'Nylon', 'Plexiglass',
        'Plexiglass', 'Polystyrene', 'Polystyrene', 'Rubber', 'Rubber', 'Brass',
        'Steel', 'Teflon', 'Teflon', 'Wood', 'Wood', 'Wood', 'Wood', 'Zinc', 'Zinc'
    ],
    'E_1': [
        200, 133, 69, 69, 115, 64, 64, 170, 248, 133, 133, 133, 60, 60, 60, 20,
        20, 170, 170, 4, 3.3, 3.3, 2.5, 2.5, 0.01, 0.01, 115, 200, 0.5, 0.5, 10,
        10, 10, 10, 82.7, 82.7
    ],
    'nu_1': [
        0.25, 0.35, 0.33, 0.33, 0.34, 0.31, 0.31, 0.26, 0.31, 0.35, 0.35, 0.35,
        0.25, 0.25, 0.25, 0.2, 0.2, 0.31, 0.31, 0.39, 0.37, 0.37, 0.4, 0.4, 0.47,
        0.47, 0.34, 0.25, 0.47, 0.47, 0.35, 0.35, 0.35, 0.35, 0.25, 0.25
    ],
    'rho_1': [
        8000, 8940, 2700, 2700, 8730, 8650, 8650, 7200, 7190, 8940, 8940, 8940,
        2400, 2400, 2400, 2050, 2050, 8900, 8900, 1130, 1190, 1190, 1040, 1040,
        2300, 2300, 8730, 8000, 2200, 2200, 750, 750, 750, 750, 7120, 7120
    ],
    'Material_2': [
        'Steel', 'Copper', 'Aluminum', 'Steel', 'Cast Iron', 'Cadmium', 'Steel',
        'Cast Iron', 'Chromium', 'Cast Iron', 'Copper', 'Steel', 'Glass', 'Steel',
        'Nickel', 'Graphite', 'Steel', 'Nickel', 'Steel', 'Nylon', 'Plexiglass',
        'Steel', 'Polystyrene', 'Steel', 'Asphalt', 'Concrete', 'Steel', 'Cast Iron',
        'Steel', 'Teflon', 'Wood', 'Copper', 'Steel', 'Concrete', 'Zinc', 'Cast Iron'
    ],
    'E_2': [
        200, 133, 69, 200, 170, 64, 200, 170, 248, 170, 133, 200, 60, 200, 170, 20,
        200, 170, 200, 4, 3.3, 200, 2.5, 200, 3, 17, 200, 170, 200, 0.5, 10, 133,
        200, 17, 82.7, 170
    ],
    'nu_2': [
        0.25, 0.35, 0.33, 0.25, 0.26, 0.31, 0.25, 0.26, 0.31, 0.26, 0.35, 0.25,
        0.25, 0.25, 0.31, 0.2, 0.25, 0.31, 0.25, 0.39, 0.37, 0.25, 0.4, 0.25, 0.35,
        0.17, 0.25, 0.26, 0.25, 0.47, 0.35, 0.35, 0.25, 0.17, 0.25, 0.26
    ],
    'rho_2': [
        8000, 8940, 2700, 8000, 7200, 8650, 8000, 7200, 7190, 7200, 8940, 8000,
        2400, 8000, 8900, 2050, 8000, 8900, 8000, 1130, 1190, 8000, 1040, 8000,
        2500, 2400, 8000, 7200, 8000, 2200, 750, 8940, 8000, 2400, 7120, 7200
    ],
    'mu_static': [
        0.5,
        0.53,
        0.48,
        0.49,
        0.57,
        0.54,
        0.53,
        0.49,
        0.79,
        0.52,
        0.56,
        0.41,
        0.41,
        0.46,
        0.44,
        0.46,
        0.55,
        0.50,
        0.49,
        0.56,
        0.41,
        0.48,
        0.51,
        0.51,
        0.46,
        0.49,
        0.47,
        0.55,
        1.46,
        0.69,
        0.46,
        0.44,
        0.46,
        0.59,
        0.50,
        0.64,
        0.73,
        0.61,
        0.55,
        0.53,
        0.54,
        0.90,
        0.64,
        0.55,
        0.55,
        0.74,
        0.54,
        0.55,
        0.41,
        0.47,
        0.51,
        0.56,
        0.55,
        0.75
    ]
}

In [ ]:
input_folder = ["/home/antoine/ServerIPC/notebooks/polyfem/scenarios/"]

In [ ]:
def run_simulations(materials, base_folder="input_files", output_folder="results"):
    """
    Run PolyFEM simulations for all material pairs, processing multiple input folders.
    
    Args:
        materials (dict): Material properties and parameters.
        base_folder (str): Folder containing subdirectories with base JSON files.
        output_folder (str): Folder to store results.
    """
    # Ensure output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # List all subdirectories in the base folder
    subdirs = [d for d in os.listdir(base_folder) if os.path.isdir(os.path.join(base_folder, d))]

    for subdir in subdirs:
        subdir_path = os.path.join(base_folder, subdir)
        json_files = [f for f in os.listdir(subdir_path) if f.endswith('.json')]

        for base_file in json_files:
            base_json_path = os.path.join(subdir_path, base_file)
            
            for i, (m1, e1, nu1, rho1, m2, e2, nu2, rho2, mu) in enumerate(zip(
                materials["Material_1"], materials["E_1"], materials["nu_1"], materials["rho_1"],
                materials["Material_2"], materials["E_2"], materials["nu_2"], materials["rho_2"],
                materials["mu_static"]
            )):
                # Load the base JSON
                with open(base_json_path, "r") as f:
                    data = json.load(f)

                # Update materials in JSON
                data["materials"] = [
                    {"id": 1, "type": "NeoHookean", "E": e1, "nu": nu1, "rho": rho1},
                    {"id": 2, "type": "NeoHookean", "E": e2, "nu": nu2, "rho": rho2}
                ]

                # Update friction coefficient
                data["contact"]["friction_coefficient"] = mu

                # Save modified JSON
                sim_output_dir = os.path.join(output_folder, subdir, os.path.splitext(base_file)[0])
                os.makedirs(sim_output_dir, exist_ok=True)
                modified_json_path = os.path.join(sim_output_dir, f"simulation_{i+1}.json")
                with open(modified_json_path, "w") as f:
                    json.dump(data, f, indent=4)

                # Run PolyFEM simulation and time it
                print(f"Running simulation for {base_file}: {m1}-{m2} pair...")
                start_time = time.time()
                result = subprocess.run(
                    ["polyfem/build/PolyFEM_bin", "-j", modified_json_path],
                    stdout=subprocess.PIPE,
                    stderr=subprocess.PIPE
                )
                elapsed_time = time.time() - start_time

                # Check simulation result
                if result.returncode == 0:
                    print(f"Simulation completed for {m1}-{m2} in {elapsed_time:.2f} seconds.")
                    
                    # Save results
                    result_path = os.path.join(sim_output_dir, f"results_{m1}_{m2}.json")
                    if os.path.exists("results.json"):  # Check if PolyFEM produces a results file
                        os.rename("results.json", result_path)
                        print(f"Results saved to {result_path}")
                    else:
                        print(f"No results.json found for {m1}-{m2}. Skipping save.")
                else:
                    # Log failure details
                    print(f"Simulation failed for {m1}-{m2} pair.")
                    error_log_path = os.path.join(sim_output_dir, f"error_{m1}_{m2}.log")
                    with open(error_log_path, "w") as f:
                        f.write(result.stderr.decode())
                    print(f"Error details saved to {error_log_path}")

    print("All simulations completed!")

In [ ]:
run_simulations(materials, base_folder="input_files", output_folder="results")